In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp traindl

# 用户的点击当做一个序列

# lib导入

In [1]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
# from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import custom_attn
import logging
import matplotlib.pyplot as plt
# import nltk
import numpy as np
import os
%matplotlib inline

In [2]:
import sys
sys.path.append('..')
from mylib.utils.pickle import PickleWrapper

In [3]:
# export
# from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gc
import tqdm
import os
from tx.config import * 
from tx.eda import * 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行
# from multiprocessing.dummy import Pool
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

In [5]:
VOCAB_FILE = os.path.join(args.DATA_DIR, 'data_gen/word2vector.model.bin')

In [6]:
# covers about 95% of input data
MAX_SENTS = 91 # maximum number of sentences per document
MAX_WORDS = 20 # maximum number of words per sentence

WORD_EMBED_SIZE = 128
SENT_EMBED_SIZE = 100
DOC_EMBED_SIZE = 50
NUM_CLASSES = 2

BATCH_SIZE = 64
NUM_EPOCHS = 10

In [7]:
wv_from_text = KeyedVectors.load_word2vec_format(VOCAB_FILE, binary=True) 

## Load Vocabulary

In [8]:
words = ['PAD'] + wv_from_text.index2word

word2id = dict(zip(words, range(len(words))))

id2word = {v:k for k, v in word2id.items()}
vocab_size = len(word2id)
print("vocab_size: {:d}".format(vocab_size))

vocab_size: 3412774


In [123]:
# PickleWrapper.dump2File(word2id, os.path.join(args.DATA_DIR, 'data_gen/creativeid2id_dict.pkl'))

## Load w2v Embeddings

In [9]:
E = np.zeros((vocab_size, WORD_EMBED_SIZE))

In [10]:
E[1:] = wv_from_text.vectors

In [11]:
E.shape

(3412774, 128)

In [12]:
# !pip install tensorflow==2.1.0
# -i https://pypi.tuna.tsinghua.edu.cn/simple

In [13]:
def pad_or_truncate(xs, maxlen):
    if len(xs) > maxlen:
        return xs[len(xs) - maxlen:]
#     return ["PAD"] * (maxlen - len(xs)) + xs
    return [0] * (maxlen - len(xs)) + xs

xs = ["The", "cat", "fought", "like", "a", "mouse"]
print(pad_or_truncate(xs, 3))
print(pad_or_truncate(xs, 7))

['like', 'a', 'mouse']
[0, 'The', 'cat', 'fought', 'like', 'a', 'mouse']


In [14]:
# rtest = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/test_creative_id.pkl'))

rtrain = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/train1_creative_id.pkl'))

In [15]:
str_cols = 'user_id age gender'.split()
user_train = pd.read_csv(os.path.join(args.DATA_DIR, 'data_origin/train_preliminary/user.csv'), converters={c: str for c in str_cols}).set_index('user_id')

In [16]:
gender2id = user_train.gender.map(lambda s: int(s)-1)
age2id = user_train.age.map(lambda s: int(s)-1)

In [65]:
user_train.head()

,age,gender
user_id,,
1,4,1
2,10,1
3,7,2
4,5,1
5,4,1


In [126]:
rtrain[0].head(2)

user_id
1     [[[1], [1], [1], [1], [1], [1], [1], [1], [1],...
10    [[[1], [1], [1], [119124, 1], [1], [1], [18, 1...
dtype: object

In [17]:
def train_batch_generator(r, batch_size=1000):
    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
    y_gender = []
    y_age = []
    cnt = 0
    while True:
        for uu in r[:-1]:
            for i, v in uu.items():
        #             print(v[0])
        #             print('--------')
                y_gender.append(gender2id[i])
                y_age.append(age2id[i])
                sents = pad_or_truncate(v[0], MAX_SENTS)
        #             print(sents)

        #             in_queue, out_queue = Manager().Queue(), Manager().Queue()

        #             for sid, words in enumerate(sents):
        #                 in_queue.put((sid, words))
        #             def f(in_queue, out_queue):

        #                 while not in_queue.empty():
        # #                     print(f'in_queue.qsize: {in_queue.qsize()}')
        #                     sid, words = in_queue.get()
        # #                     print(sid)
                for sid, words in enumerate(sents):
                    words = pad_or_truncate(words, MAX_WORDS)
        #                 print(sid)
        #                 print(words)
        #                     wordsid = pd.Series(words).map(word2id).tolist()
        #                     out_queue.put((sid, words))
        #             pool = Pool(30, f, (in_queue, out_queue))
        #             pool.close()
        #             pool.join()
        #             while not out_queue.empty():
        # #                 print(f'out_queue.qsize: {out_queue.qsize()}')
        #                 sid, wordsid = out_queue.get()
        #                 print(wordsid)
                    X[cnt, sid] = words
                cnt += 1
                if cnt == batch_size:
                    yield X, to_categorical(y_gender, num_classes=NUM_CLASSES)# y_gender, y_age
                    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
                    y_gender = []
                    y_age = []
                    cnt = 0

#                 break
                
#                 for wid, word in enumerate(words):
#                     word_id = word2id[word]
#                     X[docid, sid, wid] = word_id
            
            
            
# #             break
#         break

In [18]:
def val_batch_generator(r, batch_size=20000):
    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
    y_gender = []
    y_age = []
    cnt = 0
    while True:
        for uu in r[-1:]:
            for i, v in uu.items():
                y_gender.append(gender2id[i])
                y_age.append(age2id[i])
                sents = pad_or_truncate(v[0], MAX_SENTS)
                for sid, words in enumerate(sents):
                    words = pad_or_truncate(words, MAX_WORDS)
                    X[cnt, sid] = words
                cnt += 1
                if cnt == batch_size:
                    yield X, to_categorical(y_gender, num_classes=NUM_CLASSES)# y_gender, y_age
                    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
                    y_gender = []
                    y_age = []
                    cnt = 0

In [141]:
MAX_SENTS

91

In [142]:
a = batch_generator(rtrain, 10)

In [144]:
for X, y_gender in a:
    print(X.shape)
    print(y_gender)
    break

(10, 91, 20)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [109]:
X[1]

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.194e+03,
        1.000e+00]])

In [19]:
sent_inputs = Input(shape=(MAX_WORDS,), dtype="int32")
sent_emb = Embedding(input_dim=vocab_size,
                     output_dim=WORD_EMBED_SIZE,
                     weights=[E], 
                     trainable=False,
                     mask_zero=True
                    )(sent_inputs)

sent_enc = Bidirectional(GRU(SENT_EMBED_SIZE,
                            return_sequences=True))(sent_emb)

sent_att = custom_attn.AttentionM()(sent_enc)

sent_model = Model(inputs=sent_inputs, outputs=sent_att)
sent_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 128)           436835072 
_________________________________________________________________
bidirectional (Bidirectional (None, 20, 200)           138000    
_________________________________________________________________
attention_m (AttentionM)     (None, 200)               220       
Total params: 436,973,292
Trainable params: 138,220
Non-trainable params: 436,835,072
_________________________________________________________________


In [20]:
doc_inputs = Input(shape=(MAX_SENTS, MAX_WORDS), dtype="int32")

doc_emb = TimeDistributed(sent_model)(doc_inputs)

doc_enc = Bidirectional(GRU(DOC_EMBED_SIZE,
                           return_sequences=True))(doc_emb)

doc_att = custom_attn.AttentionM()(doc_enc)
# doc_att = AttentionM()(doc_enc)

fc1_dropout = Dropout(0.2)(doc_att)
fc1 = Dense(50, activation="relu")(fc1_dropout)
fc2_dropout = Dropout(0.2)(fc1)
doc_pred = Dense(NUM_CLASSES, activation="softmax")(fc2_dropout)

model = Model(inputs=doc_inputs, outputs=doc_pred)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 91, 20)]          0         
_________________________________________________________________
time_distributed (TimeDistri (None, 91, 200)           436973292 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 91, 100)           75600     
_________________________________________________________________
attention_m_1 (AttentionM)   (None, 100)               191       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0   

In [21]:
model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

In [23]:
history = model.fit(train_batch_generator(rtrain, 1000),
                   steps_per_epoch=880, epochs=2, validation_data=val_batch_generator(rtrain, 2000)
                    , validation_steps=10,
#                     workers=20,
#                     use_multiprocessing=True
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 880 steps, validate for 10 steps
Epoch 1/2
880/880 [==============================] - 25942s 29s/step - loss: 0.2166 - accuracy: 0.9177 - val_loss: 0.1805 - val_accuracy: 0.9341
Epoch 2/2
880/880 [==============================] - 33141s 38s/step - loss: 0.1794 - accuracy: 0.9362 - val_loss: 0.1752 - val_accuracy: 0.9367


In [24]:
model.save(os.path.join(args.DATA_DIR, 'data_gen/model_gender.h5'))

In [22]:
history = model.fit(train_batch_generator(rtrain, 1000),
                   steps_per_epoch=880, epochs=2, validation_data=val_batch_generator(rtrain, 2000)
                    , validation_steps=10,
                    workers=20,
#                     use_multiprocessing=True
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 880 steps, validate for 10 steps
Epoch 1/2
  2/880 [..............................] - ETA: 5:08:24 - loss: 0.6899 - accuracy: 0.5185

UnknownError:  RuntimeError: Your generator is NOT thread-safe. Keras requires a thread-safe generator when `use_multiprocessing=False, workers > 1`. 
Traceback (most recent call last):

  File "/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 990, in get
    inputs = self.queue.get(block=True).get()

  File "/root/anaconda3/lib/python3.7/multiprocessing/pool.py", line 657, in get
    raise self._value

  File "/root/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))

  File "/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 924, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])

ValueError: generator already executing


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", 

In [ ]:
history = model.fit(train_batch_generator(rtrain, 1000),
                   steps_per_epoch=880, epochs=2, validation_data=val_batch_generator(rtrain, 2000)
                    , validation_steps=10,
                    workers=20,
                    use_multiprocessing=True
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 880 steps, validate for 10 steps
Epoch 1/2


In [161]:
history = model.fit(train_batch_generator(rtrain, 1),
                   steps_per_epoch=2, epochs=3, validation_data=val_batch_generator(rtrain, 2), validation_steps=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 2 steps, validate for 2 steps
Epoch 1/3
2/2 [==============================] - 0s 144ms/step - loss: 0.8314 - accuracy: 0.5000 - val_loss: 0.5680 - val_accuracy: 0.7500
Epoch 2/3
2/2 [==============================] - 0s 117ms/step - loss: 0.3609 - accuracy: 1.0000 - val_loss: 0.3640 - val_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 106ms/step - loss: 0.6690 - accuracy: 0.5000 - val_loss: 0.8230 - val_accuracy: 0.2500


In [158]:
a = train_batch_generator(rtrain, 100)